**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity 
BASE DIRECTORY (FD_BASE): /mount 
REPO DIRECTORY (FD_REPO): /mount/repo 
WORK DIRECTORY (FD_WORK): /mount/work 
DATA DIRECTORY (FD_DATA): /mount/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /mount/repo/Proj_CombEffect_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /mount/repo/Proj_CombEffect_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /mount/repo/Proj_CombEffect_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /mount/repo/Proj_CombEffect_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /mount/repo/Proj_CombEffect_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /mount/repo/Proj_CombEffect_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /mount/repo/Proj_CombEffect_ENCODE_FCC/log 
PROJECT APP     (FD_APP): /mount/repo/Proj_CombEffect_ENCODE_FCC/app 
PROJECT REF     (FD_REF): /mount/repo/Proj_CombEffect_ENCODE_FCC/references 



## Import data
```
<Data>
* OL13 (FADS)
* OL43 (GATA/MYC)
* OL45 (HBE1/LMO2/RBM38/HBA2/BCL11A)

- OL13_20220512_counts.out
- OL13_20220512_normalized_counts.out
- OL43_20211228_counts.out      
- OL43_20211228_normalized_counts.out
- OL43_20221003_counts.out 
- OL43_20221003_K562_normalized_counts.out
- OL45_20220927_counts.out 
- OL45_20220927_K562_normalized_counts.out
```

**Check data**

In [2]:
fdiry = file.path(FD_DAT, "processed", "MPRA_Tiling_K562_Tewhey_hannah", "tiling_counts")
print(dir(fdiry))

 [1] "FADS_tile_snp.20190214.attributes"       
 [2] "OL13_20220512_counts.out"                
 [3] "OL13_20220512_normalized_counts.out"     
 [4] "OL43_20211228_counts.out"                
 [5] "OL43_20211228_normalized_counts.out"     
 [6] "OL43_20221003_counts.out"                
 [7] "OL43_20221003_K562_normalized_counts.out"
 [8] "OL43_K562.bed"                           
 [9] "OL43.attributes"                         
[10] "OL45_20220927_counts.out"                
[11] "OL45_20220927_K562_normalized_counts.out"
[12] "OL45_K562.bed"                           
[13] "OL45.attributes"                         
[14] "UKBB_GTEx"                               


In [3]:
fdiry = file.path(FD_DAT, "processed", "MPRA_Tiling_K562_Tewhey_hannah", "tiling_counts")
fpaths = dir(fdiry)
print(grep("out", fpaths, value=TRUE))

[1] "OL13_20220512_counts.out"                
[2] "OL13_20220512_normalized_counts.out"     
[3] "OL43_20211228_counts.out"                
[4] "OL43_20211228_normalized_counts.out"     
[5] "OL43_20221003_counts.out"                
[6] "OL43_20221003_K562_normalized_counts.out"
[7] "OL45_20220927_counts.out"                
[8] "OL45_20220927_K562_normalized_counts.out"


**Setup metadata**

In [4]:
dat_meta = data.frame(
    Dataset = c(
        "OL13_20220512", "OL13_20220512", 
        "OL43_20221003", "OL43_20221003", 
        "OL45_20220927", "OL45_20220927"),
    Process = c("raw", "norm", "raw", "norm", "raw", "norm"),
    Genome  = c("hg19", "hg19", "hg38", "hg38", "hg38", "hg38"),
    N_Rep_Input  = c(4, 4, 6, 6, 4, 4),  
    N_Rep_Output = c(4, 4, 5, 5, 4, 4), 
    FName = c(
        "OL13_20220512_counts.out",
        "OL13_20220512_normalized_counts.out",
        "OL43_20221003_counts.out",
        "OL43_20221003_K562_normalized_counts.out",
        "OL45_20220927_counts.out",
        "OL45_20220927_K562_normalized_counts.out")
)
dat_meta

Dataset       Process Genome N_Rep_Input N_Rep_Output
1 OL13_20220512 raw     hg19   4           4           
2 OL13_20220512 norm    hg19   4           4           
3 OL43_20221003 raw     hg38   6           5           
4 OL43_20221003 norm    hg38   6           5           
5 OL45_20220927 raw     hg38   4           4           
6 OL45_20220927 norm    hg38   4           4           
  FName                                   
1 OL13_20220512_counts.out                
2 OL13_20220512_normalized_counts.out     
3 OL43_20221003_counts.out                
4 OL43_20221003_K562_normalized_counts.out
5 OL45_20220927_counts.out                
6 OL45_20220927_K562_normalized_counts.out

**Read data**

In [5]:
lst_dat_read = lapply(1:nrow(dat_meta), function(idx){

    ### Extract data info from the metadata table
    xrow  = dat_meta[idx,]
    fname = xrow$FName
    n_rep_input  = xrow$N_Rep_Input
    n_rep_output = xrow$N_Rep_Output
    
    ### Set file directory
    fdiry = file.path(FD_DAT, "processed", "MPRA_Tiling_K562_Tewhey_hannah", "tiling_counts")
    fpath = file.path(fdiry, fname)
          
    ### import data
    dat = read.table(fpath, row.names=1)
    dat = dat %>% rownames_to_column(var = "Name")

    ### rename columns & assign
    cnames = c(
        "Name", 
        paste0("Input.rep",  1:n_rep_input), 
        paste0("Output.rep", 1:n_rep_output)
    )
    colnames(dat) = cnames
    
    ### show progress and return
    cat("\n=======================\n")
    cat(fname, "\n")
    cat("Shape:", dim(dat), "\n")
    print(head(dat, 3))
    flush.console()
    return(dat)
})

names(lst_dat_read) = paste(
    paste("TMPRA_K562", dat_meta$Dataset, sep="_"), 
    dat_meta$Genome, 
    dat_meta$Process, sep=".")


OL13_20220512_counts.out 
Shape: 55229 9 
                                               Name Input.rep1 Input.rep2
1       (11:61555216-61555415;11:61555315:T:C_A_wC)       1609       1221
2 (11:61555231-61555430_RC;11:61555330:T:C_A_wC_RC)       1179        582
3 (11:61555315:T:C_A_wC_RC;11:61555216-61555415_RC)       1066        643
  Input.rep3 Input.rep4 Output.rep1 Output.rep2 Output.rep3 Output.rep4
1       1396        798         845         283         574        1223
2       1225        721         476         416         912         956
3       1206        681         470         846         540         847

OL13_20220512_normalized_counts.out 
Shape: 55229 9 
                                               Name Input.rep1 Input.rep2
1       (11:61555216-61555415;11:61555315:T:C_A_wC)   881.8510  1196.3976
2 (11:61555231-61555430_RC;11:61555330:T:C_A_wC_RC)   646.1792   570.2730
3 (11:61555315:T:C_A_wC_RC;11:61555216-61555415_RC)   584.2468   630.0439
  Input.rep3 Input.rep4

## Preprocess the table: Extract only tiling regions

**Define helper functions for data processing**

In [6]:
get_strand = function(x){ifelse(is.na(x), "+", "-")}
add_chrom  = function(x){ifelse(str_detect(x, "chr"), x, paste0("chr", x))}

fun_extract = function(dat) {
    ### Input:  dataframe
    ### Output: dataframe
    ### Description:
    ###     Preprocess the raw dataframe
    ###     - expand duplicates
    ###     - filter and extract only tiling oligos
    ###     - get location information
    ### ===========================================
    ### init regex pattern
    pattern = "^[chr|1-9|X]+:[0-9]*-[0-9]*"
    
    ### preprocess
    res = dat %>% 
        dplyr::mutate(Name = gsub("\\(|\\)", "", Name)) %>% 
        tidyr::separate_rows(Name, sep=";") %>%
        dplyr::filter(!grepl(pattern = "Nadav", Name)) %>%
        dplyr::filter(str_detect(string=Name, pattern=pattern)) %>%
        tidyr::separate(Name, c("Chrom", "Start", "End", "Strand"), remove=FALSE, fill="right") %>%
        dplyr::mutate(
            Chrom  = add_chrom(Chrom),
            Start  = as.integer(Start),
            End    = as.integer(End),
            Name   = add_chrom(Name),
            Strand = get_strand(Strand))
    return(res)
}

**Apply processing functions**

In [7]:
### init
lst = lst_dat_read

### before preprocessed
print(lapply(lst, dim))
cat("++++++++++++++++++++++++++++++++++++++++\n")

### after proprocessed
lst = lapply(lst, fun_extract)
lst_dat_extract = lst
print(lapply(lst, dim))

$TMPRA_K562_OL13_20220512.hg19.raw
[1] 55229     9

$TMPRA_K562_OL13_20220512.hg19.norm
[1] 55229     9

$TMPRA_K562_OL43_20221003.hg38.raw
[1] 99307    12

$TMPRA_K562_OL43_20221003.hg38.norm
[1] 99307    12

$TMPRA_K562_OL45_20220927.hg38.raw
[1] 94381     9

$TMPRA_K562_OL45_20220927.hg38.norm
[1] 94381     9

++++++++++++++++++++++++++++++++++++++++
$TMPRA_K562_OL13_20220512.hg19.raw
[1] 43907    13

$TMPRA_K562_OL13_20220512.hg19.norm
[1] 43907    13

$TMPRA_K562_OL43_20221003.hg38.raw
[1] 96108    16

$TMPRA_K562_OL43_20221003.hg38.norm
[1] 96108    16

$TMPRA_K562_OL45_20220927.hg38.raw
[1] 71213    13

$TMPRA_K562_OL45_20220927.hg38.norm
[1] 71213    13



**Show results**

In [8]:
set.seed(123)
lst = lst_dat_extract
lapply(lst, slice_sample, n=3)

$TMPRA_K562_OL13_20220512.hg19.raw
# A tibble: 3 × 13
  Name    Chrom  Start    End Strand Input.rep1 Input.rep2 Input.rep3 Input.rep4
  <chr>   <chr>  <int>  <int> <chr>       <int>      <int>      <int>      <int>
1 chr11:… chr11 6.16e7 6.16e7 -             363        256        368        252
2 chr11:… chr11 6.16e7 6.16e7 -             823        497        638        344
3 chr11:… chr11 6.16e7 6.16e7 +             400        378        637        250
# ℹ 4 more variables: Output.rep1 <int>, Output.rep2 <int>, Output.rep3 <int>,
#   Output.rep4 <int>

$TMPRA_K562_OL13_20220512.hg19.norm
# A tibble: 3 × 13
  Name    Chrom  Start    End Strand Input.rep1 Input.rep2 Input.rep3 Input.rep4
  <chr>   <chr>  <int>  <int> <chr>       <dbl>      <dbl>      <dbl>      <dbl>
1 chr11:… chr11 6.16e7 6.16e7 -            303.       283.      305.        239.
2 chr11:… chr11 6.16e7 6.16e7 +            489.       551.      591.        540.
3 chr11:… chr11 6.17e7 6.17e7 +            119.       118.       86.4       141.
# ℹ 4 more variables: Output.rep1 <dbl>, Output.rep2 <dbl>, Output.rep3 <dbl>,
#   Output.rep4 <dbl>

$TMPRA_K562_OL43_20221003.hg38.raw
# A tibble: 3 × 16
  Name    Chrom  Start    End Strand Input.rep1 Input.rep2 Input.rep3 Input.rep4
  <chr>   <chr>  <int>  <int> <chr>       <int>      <int>      <int>      <int>
1 chr8:1… chr8  1.27e8 1.27e8 +             498        576        544        596
2 chrX:4… chrX  4.85e7 4.85e7 +             271        290        221        253
3 chrX:4… chrX  4.83e7 4.83e7 +             181        221        277        253
# ℹ 7 more variables: Input.rep5 <int>, Input.rep6 <int>, Output.rep1 <int>,
#   Output.rep2 <int>, Output.rep3 <int>, Output.rep4 <int>, Output.rep5 <int>

$TMPRA_K562_OL43_20221003.hg38.norm
# A tibble: 3 × 16
  Name    Chrom  Start    End Strand Input.rep1 Input.rep2 Input.rep3 Input.rep4
  <chr>   <chr>  <int>  <int> <chr>       <dbl>      <dbl>      <dbl>      <dbl>
1 chr8:1… chr8  1.27e8 1.27e8 +            234.       222.       171.       221.
2 chr8:1… chr8  1.27e8 1.27e8 +            438.       409.       412.       391.
3 chr8:1… chr8  1.28e8 1.28e8 +           1120.      1169.      1135.      1183.
# ℹ 7 more variables: Input.rep5 <dbl>, Input.rep6 <dbl>, Output.rep1 <dbl>,
#   Output.rep2 <dbl>, Output.rep3 <dbl>, Output.rep4 <dbl>, Output.rep5 <dbl>

$TMPRA_K562_OL45_20220927.hg38.raw
# A tibble: 3 × 13
  Name    Chrom  Start    End Strand Input.rep1 Input.rep2 Input.rep3 Input.rep4
  <chr>   <chr>  <int>  <int> <chr>       <int>      <int>      <int>      <int>
1 chr11:… chr11 3.38e7 3.38e7 +             786        596        647        773
2 chr11:… chr11 5.57e6 5.57e6 +             294        254        226        187
3 chr2:6… chr2  6.00e7 6.00e7 +             863        780        827        797
# ℹ 4 more variables: Output.rep1 <int>, Output.rep2 <int>, Output.rep3 <int>,
#   Output.rep4 <int>

$TMPRA_K562_OL45_20220927.hg38.norm
# A tibble: 3 × 13
  Name    Chrom  Start    End Strand Input.rep1 Input.rep2 Input.rep3 Input.rep4
  <chr>   <chr>  <int>  <int> <chr>       <dbl>      <dbl>      <dbl>      <dbl>
1 chr11:… chr11 3.37e7 3.37e7 +            711.       672.       618.       669.
2 chr11:… chr11 3.42e7 3.42e7 +            129.       118.       126.       141.
3 chr11:… chr11 3.35e7 3.35e7 +            525.       536.       466.       634.
# ℹ 4 more variables: Output.rep1 <dbl>, Output.rep2 <dbl>, Output.rep3 <dbl>,
#   Output.rep4 <dbl>

## Filter out the fragments (Oligos) where mean input is zero

**Define helper functions**

In [9]:
fun_filter = function(dat){
    ### Input:  dataframe
    ### Output: dataframe
    ### Description:
    ###     calculate the mean of input for each fragment and 
    ###     filter out the fragment w/ plasmid mean is zero
    ###     - convert wide matrix to long matrix
    ###     - group by the input and output to calculate the mean
    ###     - get the non-zero fragments from the input dataframe
    
    ### calculate the mean of input for each fragment and 
    tmp = dat %>% 
        tidyr::gather(Sample, Value, -Chrom, -Start, -End, -Name, -Strand) %>% 
        tidyr::separate(Sample, c("Group", "Rep"), sep="\\.", remove=FALSE) %>%
        dplyr::filter(Group == "Input") %>%
        dplyr::group_by(Chrom, Start, End, Name, Strand, Group) %>%
        dplyr::summarise(Value = mean(Value), .groups="drop")
    
    ### filter out the fragment w/ plasmid mean is zero
    tmp = tmp %>% dplyr::filter(Value > 0)
    idx = unique(tmp$Name)
    res = dat %>% dplyr::filter(Name %in% idx)
    return(res)
}

**Filter fragments**

In [10]:
### init
lst = lst_dat_extract

### show info before processing
print(lapply(lst, dim))
cat("++++++++++++++++++++++++++++++++++++++++\n")

### processing
lst = lapply(lst, fun_filter)

### assign and show info
lst_dat_filter = lst
print(lapply(lst, dim))

$TMPRA_K562_OL13_20220512.hg19.raw
[1] 43907    13

$TMPRA_K562_OL13_20220512.hg19.norm
[1] 43907    13

$TMPRA_K562_OL43_20221003.hg38.raw
[1] 96108    16

$TMPRA_K562_OL43_20221003.hg38.norm
[1] 96108    16

$TMPRA_K562_OL45_20220927.hg38.raw
[1] 71213    13

$TMPRA_K562_OL45_20220927.hg38.norm
[1] 71213    13

++++++++++++++++++++++++++++++++++++++++
$TMPRA_K562_OL13_20220512.hg19.raw
[1] 43871    13

$TMPRA_K562_OL13_20220512.hg19.norm
[1] 43871    13

$TMPRA_K562_OL43_20221003.hg38.raw
[1] 96087    16

$TMPRA_K562_OL43_20221003.hg38.norm
[1] 96087    16

$TMPRA_K562_OL45_20220927.hg38.raw
[1] 71200    13

$TMPRA_K562_OL45_20220927.hg38.norm
[1] 71200    13



**Show results**

In [11]:
lst = lst_dat_filter
dat = lst[[1]]
head(dat, 3)

Name                       Chrom Start    End      Strand Input.rep1
1 chr11:61555216-61555415    chr11 61555216 61555415 +      1609      
2 chr11:61555231-61555430_RC chr11 61555231 61555430 -      1179      
3 chr11:61555216-61555415_RC chr11 61555216 61555415 -      1066      
  Input.rep2 Input.rep3 Input.rep4 Output.rep1 Output.rep2 Output.rep3
1 1221       1396       798        845         283         574        
2  582       1225       721        476         416         912        
3  643       1206       681        470         846         540        
  Output.rep4
1 1223       
2  956       
3  847

## Calculate mean for normalized count table

**Define helper functions**

In [12]:
fun_add_mean_column = function(dat){
    ### get input and output to calculate log2FC
    x_inp = dat %>% dplyr::select(starts_with("Input"))  %>% apply(., 1, mean)
    x_out = dat %>% dplyr::select(starts_with("Output")) %>% apply(., 1, mean)
    x_lfc = log2(x_out) - log2(x_inp)
    
    ### add columns
    dat$Input.mean  = x_inp
    dat$Output.mean = x_out
    dat$Log2FC.mean = x_lfc
 
    return(dat)
}

**Calculate mean and add it as an additional column for normalized count**

In [13]:
lst = lst_dat_filter
print(names(lst))
cat("=======================\n")
sid = grep("norm", names(lst), value=TRUE)
print(sid)

[1] "TMPRA_K562_OL13_20220512.hg19.raw"  "TMPRA_K562_OL13_20220512.hg19.norm"
[3] "TMPRA_K562_OL43_20221003.hg38.raw"  "TMPRA_K562_OL43_20221003.hg38.norm"
[5] "TMPRA_K562_OL45_20220927.hg38.raw"  "TMPRA_K562_OL45_20220927.hg38.norm"
[1] "TMPRA_K562_OL13_20220512.hg19.norm" "TMPRA_K562_OL43_20221003.hg38.norm"
[3] "TMPRA_K562_OL45_20220927.hg38.norm"


In [14]:
### init
lst  = lst_dat_filter
idxs = grep("norm", names(lst), value=TRUE)

### add mean for normalization count
for (idx in idxs){
    cat(idx, "\n")
    lst[[idx]] = fun_add_mean_column(lst[[idx]])
}

### assigned
lst_dat_prep = lst

TMPRA_K562_OL13_20220512.hg19.norm 
TMPRA_K562_OL43_20221003.hg38.norm 
TMPRA_K562_OL45_20220927.hg38.norm 


**Show results**

In [15]:
lst = lst_dat_prep
idx = names(lst)[1]
dat = lst_dat_prep[[1]]

cat(idx)
head(dat, 3)

TMPRA_K562_OL13_20220512.hg19.raw

Name                       Chrom Start    End      Strand Input.rep1
1 chr11:61555216-61555415    chr11 61555216 61555415 +      1609      
2 chr11:61555231-61555430_RC chr11 61555231 61555430 -      1179      
3 chr11:61555216-61555415_RC chr11 61555216 61555415 -      1066      
  Input.rep2 Input.rep3 Input.rep4 Output.rep1 Output.rep2 Output.rep3
1 1221       1396       798        845         283         574        
2  582       1225       721        476         416         912        
3  643       1206       681        470         846         540        
  Output.rep4
1 1223       
2  956       
3  847

In [16]:
lst = lst_dat_prep
idx = names(lst)[2]
dat = lst_dat_prep[[2]]

cat(idx)
head(dat, 3)

TMPRA_K562_OL13_20220512.hg19.norm

Name                       Chrom Start    End      Strand Input.rep1
1 chr11:61555216-61555415    chr11 61555216 61555415 +      881.8510  
2 chr11:61555231-61555430_RC chr11 61555231 61555430 -      646.1792  
3 chr11:61555216-61555415_RC chr11 61555216 61555415 -      584.2468  
  Input.rep2 Input.rep3 Input.rep4 Output.rep1 Output.rep2 Output.rep3
1 1196.3976  855.5317   774.9257   1807.947     569.6449    730.4747  
2  570.2730  750.7352   700.1522   1018.441     837.3579   1160.6148  
3  630.0439  739.0912   661.3088   1005.604    1702.8961    687.2061  
  Output.rep4 Input.mean Output.mean Log2FC.mean
1 1493.445    927.1765   1150.378    0.3111919  
2 1167.403    666.8349   1045.954    0.6494180  
3 1034.299    653.6727   1107.501    0.7606680

In [17]:
lst = lst_dat_prep
idx = names(lst)[3]
dat = lst_dat_prep[[3]]

cat(idx)
head(dat, 3)

TMPRA_K562_OL43_20221003.hg38.raw

Name                     Chrom Start     End       Strand Input.rep1
1 chr8:127381651-127381850 chr8  127381651 127381850 +      1318      
2 chr8:127320401-127320600 chr8  127320401 127320600 +      1318      
3 chr8:127742001-127742200 chr8  127742001 127742200 +       634      
  Input.rep2 Input.rep3 Input.rep4 Input.rep5 Input.rep6 Output.rep1
1 1810       1477       1505       738        687        620        
2 1810       1477       1505       738        687        620        
3  730        698        761       316        343        158        
  Output.rep2 Output.rep3 Output.rep4 Output.rep5
1 636         781         520         538        
2 636         781         520         538        
3 194         268         141         251

In [18]:
lst = lst_dat_prep
idx = names(lst)[4]
dat = lst_dat_prep[[4]]

cat(idx)
head(dat, 3)

TMPRA_K562_OL43_20221003.hg38.norm

Name                     Chrom Start     End       Strand Input.rep1
1 chr8:127381651-127381850 chr8  127381651 127381850 +      887.4466  
2 chr8:127320401-127320600 chr8  127320401 127320600 +      887.4466  
3 chr8:127742001-127742200 chr8  127742001 127742200 +      426.8901  
  Input.rep2 Input.rep3 Input.rep4 Input.rep5 Input.rep6 Output.rep1
1 1078.7715  946.8314   945.7219   962.3235   893.9085   961.7725   
2 1078.7715  946.8314   945.7219   962.3235   893.9085   961.7725   
3  435.0846  447.4531   478.2022   412.0518   446.3037   245.0969   
  Output.rep2 Output.rep3 Output.rep4 Output.rep5 Input.mean Output.mean
1 1136.6673   842.9706    896.6654    760.1443    952.5006   919.6440   
2 1136.6673   842.9706    896.6654    760.1443    952.5006   919.6440   
3  346.7193   289.2652    243.1343    354.6398    440.9976   295.7711   
  Log2FC.mean
1 -0.0506444 
2 -0.0506444 
3 -0.5762898

## Store the fragments and counts
```
<Data>
* OL13 (FADS)
* OL43 (GATA/MYC)
* OL45 (HBE1/LMO2/RBM38/HBA2/BCL11A)

- OL13_20220512_counts.out
- OL13_20220512_normalized_counts.out
- OL43_20211228_counts.out      
- OL43_20211228_normalized_counts.out
- OL43_20221003_counts.out 
- OL43_20221003_K562_normalized_counts.out
- OL45_20220927_counts.out 
- OL45_20220927_K562_normalized_counts.out

<Results>
PROJECT/results/assay_fcc
└── MPRA_Tiling_K562_Tewhey_hannah
    ├── fragment_counts
    │   └── summary
    │       ├── TMPRA_K562_OL13_20220512.hg19.raw.stranded_pos.tsv 
    │       ├── TMPRA_K562_OL13_20220512.hg19.norm.stranded_pos.tsv 
    │       ├── TMPRA_K562_OL43_20221003.hg38.raw.stranded_pos.tsv 
    │       ├── TMPRA_K562_OL43_20221003.hg38.norm.stranded_pos.tsv 
    │       ├── TMPRA_K562_OL45_20220927.hg38.raw.stranded_pos.tsv 
    │       └── TMPRA_K562_OL45_20220927.hg38.norm.stranded_pos.tsv 
    │
    └── fragment
        ├── TMPRA_K562_OL13_20220512.hg19.raw.stranded_pos.bed.gz 
        ├── TMPRA_K562_OL43_20221003.hg38.raw.stranded_pos.bed.gz 
        └── TMPRA_K562_OL45_20220927.hg38.raw.stranded_pos.bed.gz 
```

### Setup output directory

In [19]:
### init
TXT_GROUP   = "assay_fcc"
TXT_ASSAY   = "MPRA_Tiling_K562_Tewhey_hannah"

### create fragment folder
txt_folder = "fragment"
txt_fdiry = file.path(FD_RES, TXT_GROUP, TXT_ASSAY, txt_folder)
cat(txt_fdiry, "\n")

txt_cmd = paste("mkdir -p", txt_fdiry)
system(txt_cmd)

### create fragment count folders
txt_folder = "fragment_counts/summary"
txt_fdiry = file.path(FD_RES, TXT_GROUP, TXT_ASSAY, txt_folder)
cat(txt_fdiry, "\n")

txt_cmd = paste("mkdir -p", txt_fdiry)
system(txt_cmd)

/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/MPRA_Tiling_K562_Tewhey_hannah/fragment 
/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/MPRA_Tiling_K562_Tewhey_hannah/fragment_counts/summary 


In [20]:
### check loop names
lst  = lst_dat_prep
idxs = grep("raw", names(lst), value=TRUE)
for (idx in idxs) {cat(idx, "\n")}

TMPRA_K562_OL13_20220512.hg19.raw 
TMPRA_K562_OL43_20221003.hg38.raw 
TMPRA_K562_OL45_20220927.hg38.raw 


### Store fragments

In [21]:
txt_group  = TXT_GROUP
txt_assay  = TXT_ASSAY
txt_folder = "fragment"
txt_strand = "stranded_pos"

lst  = lst_dat_prep
idxs = grep("raw", names(lst), value=TRUE)

txt_fdiry = file.path(FD_RES, txt_group, txt_assay, txt_folder)
cat("Output directory:", "\n")
cat(txt_fdiry, "\n")

for (idx in idxs){
    ### show progres
    cat("\n=======================")
    cat("\nSample:", idx, "\n")
    
    ### init
    dat = lst[[idx]]
    
    ### extract positive trands and order by positions
    dat = dat %>% 
        dplyr::filter(Strand == "+") %>% 
        dplyr::arrange(Chrom, Start, End)
    
    ### arrange the column into bed file format
    dat = dat %>%
        dplyr::mutate(Score = ".") %>%
        dplyr::select(Chrom, Start, End, Name, Score, Strand)
    
    ### save table
    txt_fname = paste(idx, txt_strand, "bed.gz", sep=".")
    txt_fpath = file.path(txt_fdiry, txt_fname)
    write_tsv(dat, txt_fpath, col_names=FALSE)
    
    ### show progress
    #print(fpath)
    print(head(dat))
    cat("\nSaved Table:", txt_fname, "\n")
}

Output directory: 
/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/MPRA_Tiling_K562_Tewhey_hannah/fragment 

Sample: TMPRA_K562_OL13_20220512.hg19.raw 
# A tibble: 6 × 6
  Chrom    Start      End Name                    Score Strand
  <chr>    <int>    <int> <chr>                   <chr> <chr> 
1 chr11 61554801 61555000 chr11:61554801-61555000 .     +     
2 chr11 61554806 61555005 chr11:61554806-61555005 .     +     
3 chr11 61554811 61555010 chr11:61554811-61555010 .     +     
4 chr11 61554816 61555015 chr11:61554816-61555015 .     +     
5 chr11 61554821 61555020 chr11:61554821-61555020 .     +     
6 chr11 61554826 61555025 chr11:61554826-61555025 .     +     

Saved Table: TMPRA_K562_OL13_20220512.hg19.raw.stranded_pos.bed.gz 

Sample: TMPRA_K562_OL43_20221003.hg38.raw 
# A tibble: 6 × 6
  Chrom     Start       End Name                     Score Strand
  <chr>     <int>     <int> <chr>                    <chr> <chr> 
1 chr8  126735901 126736100 chr8:126735901-126736100 .

### Store table of counts

In [22]:
txt_group  = TXT_GROUP
txt_assay  = TXT_ASSAY
txt_folder = "fragment_counts/summary"
txt_strand = "stranded_pos"

lst  = lst_dat_prep
idxs = names(lst)

txt_fdiry = file.path(FD_RES, txt_group, txt_assay, txt_folder)
cat("Output directory:", "\n")
cat(txt_fdiry, "\n")

for (idx in idxs){
    ### show progres
    cat("\n=======================")
    cat("\nSample:", idx, "\n")
    
    ### init
    dat = lst[[idx]]
    
    ### extract positive trands and order by positions
    dat = dat %>% 
       dplyr::filter(Strand == "+") %>% 
       dplyr::arrange(Chrom, Start, End)
    
    ### save table
    txt_fname = paste(idx, txt_strand, "tsv", sep=".")
    txt_fpath = file.path(txt_fdiry, txt_fname)
    write_tsv(dat, txt_fpath)

    ### show progress
    print(head(dat))
    cat("\nSaved Table:", txt_fname, "\n")
}

Output directory: 
/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/MPRA_Tiling_K562_Tewhey_hannah/fragment_counts/summary 

Sample: TMPRA_K562_OL13_20220512.hg19.raw 
# A tibble: 6 × 13
  Name    Chrom  Start    End Strand Input.rep1 Input.rep2 Input.rep3 Input.rep4
  <chr>   <chr>  <int>  <int> <chr>       <int>      <int>      <int>      <int>
1 chr11:… chr11 6.16e7 6.16e7 +             971        542        785        640
2 chr11:… chr11 6.16e7 6.16e7 +            1267        562       1059        564
3 chr11:… chr11 6.16e7 6.16e7 +            1183        641       1118        655
4 chr11:… chr11 6.16e7 6.16e7 +            1020        476       1086        584
5 chr11:… chr11 6.16e7 6.16e7 +            1138        580        894        565
6 chr11:… chr11 6.16e7 6.16e7 +             702        406        745        318
# ℹ 4 more variables: Output.rep1 <int>, Output.rep2 <int>, Output.rep3 <int>,
#   Output.rep4 <int>

Saved Table: TMPRA_K562_OL13_20220512.hg19.raw.stranded